In [7472]:
import pandas as pd
import datetime

In [7473]:
def get_SystSampl( TotalCount, flRate = 0.45):
    if type(TotalCount) == int:
         return round(TotalCount*flRate)
    else:
        intVar = TotalCount.shape[0]
        return round(intVar*flRate)

In [7475]:
data = pd.read_csv('RTG_CSV.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)

In [7476]:
data.loc[data.gender == 'М', "gender"] = 'M'
data.loc[data.gender == 'Ж', "gender"] = 'F'

In [7477]:
data

,client_id,gender,birth_date,create_date,nonresident_flag,city,contract_sum,product_category_name,card_id,card_type_name,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum,card_type
0,fe60b594364f9f636266ed1ef4f89c32,F,1985,2020-01-29,R,Гусь-Хрустальный,0,Договор на текущий счет для дебетовой карты,a83cbc33f164deae41b3339141923cc7,Visa Platinum Rewards,2019,2021,16600,71,39700,25700,220600,201000,dc
1,3012cabca5885ed53d348d6e57dab5de,M,1951,2009-09-24,R,Ишимбай,0,Договор на текущий счет для дебетовой карты,8dae17a830bb05d151034df618efbcad,VISA Classic,2019,-1,4900,2,38000,26800,56300,31300,dc
2,d871ef96820b6c7a1ada8e01a772724e,F,1982,2006-12-25,R,Юрга,0,Договор на текущий счет для дебетовой карты,429ca8c43cdc5f3d8c9efc9069ee3fa6,NaN,-1,-1,0,0,0,0,0,0,dc
3,0bacbc2abd314f9fac031d249e485a7b,M,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7f4adb9187b867ebea15503298437a3,Visa Rewards,2021,-1,47800,12,0,0,51500,51500,cc
4,0bacbc2abd314f9fac031d249e485a7b,M,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7d13882154e965dca7070eb8d626f77,Eurocard/MasterCard World,2018,2022,47800,12,0,0,51500,51500,cc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16308,9acb18399b830419bac9653b52754646,M,1986,2015-05-20,R,Уфа,0,Договор на текущий счет для дебетовой карты,d090cb20c22e2643ec366daf8d26d646,NaN,-1,-1,0,0,0,0,0,0,dc
16309,8b3206ca8d77aa2446824971f47c531b,F,1996,2022-04-28,R,Казань,30000,Кредитная карта,5a0041efbd5832765caf8b9f861bffa3,NaN,-1,-1,22700,7,0,0,22700,22700,dc
16310,d69d3201104ab393c0638839a7284ad7,M,1953,2018-08-08,R,Пятигорск,200000,Кредитная карта,a78c73e84e893179de17071583cf5084,NaN,-1,-1,0,0,0,300,9900,10100,dc
16311,a936fb30e2268d2874cdcaa221032915,M,1997,2022-05-12,R,Кемерово,15000,Кредитная карта,e3304eaf55891a175b6abf8030b99da1,NaN,-1,-1,15000,33,0,0,15000,15000,dc


Мужчины / Женщины

In [7478]:
ManToWoman = data
ManToWoman = ManToWoman.drop_duplicates(subset=['client_id'])
ManToWoman = ManToWoman.sample(n=get_SystSampl(ManToWoman))
ManToWoman = ManToWoman['gender'].value_counts()

In [7481]:
ManToWoman = ManToWoman.to_frame()
ManToWoman = ManToWoman.rename(columns = {0 : 'Gender'})

In [7482]:
ManToWoman

,gender
F,2886
M,2539


In [7483]:
ManToWoman.to_json('ManToWoman.json', index='Gender', indent=1)

Количество приобретённых карт за год

In [7484]:
now = datetime.datetime.now()

In [7485]:
cardTypeDF = data.query(f"start_date >= {now.year}-1")
cardTypeDF = cardTypeDF.sample(n=get_SystSampl(cardTypeDF))

In [7486]:
listCardType = cardTypeDF['card_type_name'].unique()

In [7543]:
cardTypeDF = cardTypeDF['card_type_name'].value_counts()
cardTypeDF

KeyError: 'card_type_name'

In [7488]:
sum = cardTypeDF.values.sum()

In [7490]:
listCreditSys = ['МИР', 'MIR', 'Visa', 'MasterCard','Maestro','Other']
listCreditSysCount = [0]*len(listCreditSys)

In [7491]:
for i in range(len(cardTypeDF.index)):
    for j in range(len(listCreditSys)):
        if (listCreditSys[j] in cardTypeDF.index[i]) or (listCreditSys[j] in cardTypeDF.index[i].capitalize()) : listCreditSysCount[j] += cardTypeDF.values[i]

In [7492]:
var = 0
for i in range(len(listCreditSysCount)): var += listCreditSysCount[i]

In [7493]:
listCreditSysCount[len(listCreditSysCount[:-1])] = sum - var

In [7494]:
listCreditSys = listCreditSys[1:]
listCreditSysCount[1] = listCreditSysCount[0] + listCreditSysCount[1]
listCreditSysCount = listCreditSysCount[1:]

In [7495]:
for i in range(len(listCreditSysCount)): listCreditSysCount[i] = round(listCreditSysCount[i]*100/sum, 1)

In [7496]:
series_obj = pd.Series(listCreditSysCount, index=listCreditSys)

In [7497]:
NewBankCard = series_obj.to_frame()
NewBankCard = NewBankCard.rename(columns = {0:'System_name'})

In [7498]:
NewBankCard

,System_name
MIR,27.3
Visa,58.0
MasterCard,13.6
Maestro,0.0
Other,1.2


In [7499]:
NewBankCard.to_json('NewBankCard.json', indent=1)

Кол-во действующих банковских карт у человека в среднем

In [7500]:
ValueCardData = data
ValueCardData = ValueCardData.query("start_date != -1 & fact_close_date == -1")
ValueCardData = ValueCardData.drop(['purchase_sum','create_date', "create_date",'purchase_count','current_balance_avg_sum','current_balance_sum','current_debit_turn_sum','current_credit_turn_sum','contract_sum','gender','birth_date'], axis=1)
ValueCardData = ValueCardData.drop(['card_type_name', 'start_date', 'fact_close_date', 'card_type', 'product_category_name', 'city', 'nonresident_flag'], axis=1)

In [7501]:
ValueCardData = ValueCardData.value_counts('client_id')

In [7502]:
ValueCardData = ValueCardData.drop_duplicates()
ValueCardData = ValueCardData.sample(n=get_SystSampl(ValueCardData))

In [7503]:
listCreditCard = [1,2,3,"4+"]
listCreditCardCount = [0]*len(listCreditCard)

In [7504]:
for i in range(len(ValueCardData.index)):
        if ValueCardData[i] == 1:
            listCreditCardCount[0] +=1
        elif ValueCardData[i] == 2:
            listCreditCardCount[1] +=1
        elif ValueCardData[i] == 3:
            listCreditCardCount[2] +=1
        else:
            listCreditCardCount[3] +=1

In [7505]:
intSum = 0
for i in range(len(listCreditCardCount)) : intSum += listCreditCardCount[i]

In [7506]:
for i in range(len(listCreditCardCount)):
    listCreditCardCount[i] = round(listCreditCardCount[i] * 100 / intSum,2)

In [7507]:
list_tuples  = list(zip(listCreditCard, listCreditCardCount))
NewBankCard = pd.DataFrame(list_tuples, columns=['CardType', 'CardCount'])

In [7508]:
NewBankCard

,CardType,CardCount
0,1,25.0
1,2,0.0
2,3,0.0
3,4+,75.0


In [7509]:
NewBankCard.to_json('CreditCardCount.json',index='CardType', indent=1)

Возраст пользователей 

In [7510]:
ValueAgeData = data.query("start_date != -1 & fact_close_date == -1")
ValueAgeData = ValueAgeData.drop(['purchase_sum','create_date', "create_date",'purchase_count','current_balance_avg_sum','current_balance_sum','current_debit_turn_sum','current_credit_turn_sum','contract_sum','gender'], axis=1)
ValueAgeData = ValueAgeData.drop(['card_type_name', 'start_date', 'fact_close_date', 'card_type', 'product_category_name', 'city', 'nonresident_flag','card_id'], axis=1)
ValueAgeData = ValueAgeData.sample(n=get_SystSampl(ValueAgeData))

In [7511]:
ValueAgeData.query(f"{now.year-14} <= birth_date ").shape[0]

1

In [7512]:
def getlistEqualPartsOfNumberParts(intNub, intParts):
  d, r = divmod(intNub, intParts)
  return [d + (1 if i < r else 0) for i in range(intParts)]

In [7513]:
intMaxAge = 66

In [7514]:
listAgeZone = getlistEqualPartsOfNumberParts(100,14)
listAgeZone

[8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

In [7515]:
listCountAgeZone = [0] * len(listAgeZone)
listSrtAgeZone = []
if listAgeZone[0] < 14: intLeft = 14-listAgeZone[0]
intRigth = listAgeZone[0]
intSum = 0

In [7516]:
for i in range(len(listAgeZone)-1):
    intLeft += listAgeZone[i]
    intRigth += listAgeZone[i+1]
    listCountAgeZone[i] += ValueAgeData.query(f"{now.year-intLeft} >= birth_date > {now.year-intRigth}").shape[0]
    intSum += listCountAgeZone[i]
    listSrtAgeZone.append( str(str(intLeft) + '-' + str(intRigth)) )
    if i == 0 : intLeft -= 14-listAgeZone[0]


In [7517]:
listCountAgeZone

[0, 245, 396, 649, 681, 553, 395, 315, 123, 46, 8, 4, 0, 0]

In [7518]:
for i in range(len(listCountAgeZone)): listCountAgeZone[i] = round(listCountAgeZone[i]   * 100 / intSum,2)

In [7519]:
listSrtAgeZone

['14-16',
 '16-23',
 '23-30',
 '30-37',
 '37-44',
 '44-51',
 '51-58',
 '58-65',
 '65-72',
 '72-79',
 '79-86',
 '86-93',
 '93-100']

In [7520]:
intSum

3415

In [7521]:
#intCountOne   = ValueAgeData.query(f"{now.year-18} >= birth_date >= {now.year-30}").shape[0]
#intCountTwo   = ValueAgeData.query(f"{now.year-31} >= birth_date >= {now.year-42}").shape[0]
#intCountThree   = ValueAgeData.query(f"{now.year-43} >= birth_date >= {now.year-55}").shape[0]
#intCountFour = ValueAgeData.query(f"{now.year-67} > birth_date").shape[0]
#intSizeValueAgeData = ValueAgeData.shape[0]

In [7522]:
#[intCountOne,intCountTwo,intCountThree,intCountFour]

In [7523]:
#intCountOne   = round(intCountOne   * 100 / intSizeValueAgeData,2)
#intCountTwo   = round(intCountTwo   * 100 / intSizeValueAgeData,2)
#intCountThree = round(intCountThree * 100 / intSizeValueAgeData,2)
#intCountFour = round(intCountFour  * 100 / intSizeValueAgeData,2)

In [7524]:
list_tuples  = list(zip(listSrtAgeZone[:-1], listCountAgeZone ))
OldProcent = pd.DataFrame(list_tuples, columns=['Age','Prosent']) 

In [7525]:
OldProcent

,Age,Prosent
0,14-16,0.00
1,16-23,7.17
2,23-30,11.60
3,30-37,19.00
4,37-44,19.94
5,44-51,16.19
6,51-58,11.57
7,58-65,9.22
8,65-72,3.60
9,72-79,1.35


In [7526]:
OldProcent.to_json('OldProcent.json', index='Age', indent=1)

Сколько средний остаток за месяц по кредитным картам

In [7527]:
turnBalanceData = data
turnBalanceData = turnBalanceData.drop([ 'gender', 'birth_date', 'create_date', 'nonresident_flag', 'city', 'contract_sum', 'card_type_name', 'start_date', 'fact_close_date'], axis=1)
balanceData = turnBalanceData.drop_duplicates(subset='card_id')
creditTurnBalanceData = turnBalanceData.query("current_credit_turn_sum > 0 & product_category_name == 'Договор на текущий счет для дебетовой карты'")
creditTurnBalanceData = creditTurnBalanceData.sample(n=get_SystSampl(creditTurnBalanceData))
debetTurnBalanceData  = turnBalanceData.query("current_debit_turn_sum > 0  & product_category_name == 'Кредитная карта'")
debetTurnBalanceData = debetTurnBalanceData.sample(n=get_SystSampl(debetTurnBalanceData))
#ValueAgeData = ValueAgeData.sample(n=get_SystSampl(ValueAgeData))

In [7528]:
[debetTurnBalanceData.shape[0], creditTurnBalanceData.shape[0]]

[699, 4905]

In [7529]:
medianDebetTurnBalanceData = debetTurnBalanceData['current_debit_turn_sum'].median()
medianCreditTurnBalanceData = creditTurnBalanceData['current_credit_turn_sum'].median()

In [7530]:
list_tuples  = list(zip(['Median debet turn sum','Median credit turn sum'], [int(medianDebetTurnBalanceData), int(medianCreditTurnBalanceData)]))
medianBalance = pd.DataFrame(list_tuples, columns=['Type','Amount']) 

In [7531]:
medianBalance

,Type,Amount
0,Median debet turn sum,42700
1,Median credit turn sum,44000


In [7532]:
medianBalance.to_json('MedianBalance.json', index='Type', indent=1)

Среднее время пользование банком

In [7534]:
AvrTimeUse = data
AvrTimeUse = AvrTimeUse[ ['client_id', 'card_id','start_date', 'fact_close_date'] ]
AvrTimeUse = AvrTimeUse.query("start_date > 0").sample(n=get_SystSampl(AvrTimeUse))
AvrTimeUse['Sum'] = 0
AvrTimeUse.loc[((AvrTimeUse.fact_close_date == -1)  ), "fact_close_date"] = now.year
AvrTimeUse.Sum = AvrTimeUse.fact_close_date - AvrTimeUse.start_date

In [7541]:
medianAvrTimeUse = AvrTimeUse.Sum.median()

In [7542]:
medianAvrTimeUse

1.0